<h1> Advanced Learning for Text and Graph Data </h1>
<b> Université Paris-Saclay - Master M2 Data Science - February/March 2017</b> <br>
<i> Students : Peter Martigny & Mehdi Miah </i> <br>

# Third part  : predict and measure the error

## Open the data

In [45]:
#our own modules
from map_tools import * #functions computing mean average precision
from handy_structures import *

#common libraries
import pandas as pd
import numpy as np
import time
import random
import pandas as pd
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
%matplotlib inline

#for words analysis
import nltk
import string
import os

from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+') #to remove punctuations

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
from sklearn.metrics.pairwise import cosine_similarity

In [46]:
def stem_tokens(tokens):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens)
    return stems
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')

In [2]:
#path to data and intermediate results
path_to_data = "..\\data\\"
path_to_results = "..\\results\\"

## == open files ====

#original data
training_set = pd.read_csv(path_to_data + 'training_set.csv')
training_info = pd.read_csv(path_to_data + 'training_info.csv', 
                            dtype = {'mid': object, 'date': object, 'body': object, 'recipients' : object})
test_set = pd.read_csv(path_to_data + 'test_set.csv')
test_info = pd.read_csv(path_to_data + 'test_info.csv',
                        dtype = {'mid': object, 'date': object, 'body': object, 'recipients' : object})

#intermediate data
training_df = pd.read_csv(path_to_results + 'training_df.csv',
                          dtype = {'mid': object, 'sender': object, 'date': object, 'body': object, 'recipients' : object})
test_df = pd.read_csv(path_to_results + 'test_df.csv',
                          dtype = {'mid': object, 'sender': object, 'date': object, 'body': object})

In [3]:
training_set.head()

,sender,mids
0,karen.buckley@enron.com,158713 158697 200301 158679 278595 298162 2002...
1,amr.ibrahim@enron.com,215241 3437 215640 3506 191790 3517 3520 3562 ...
2,andrea.ring@enron.com,270705 270706 270707 270708 270709 270710 2707...
3,sylvia.hu@enron.com,111444 111422 183084 111412 111347 110883 1105...
4,phillip.platter@enron.com,327074 327384 327385 264443 274124 274125 2741...


In [4]:
training_info.head()

,mid,date,body,recipients
0,60,2000-07-25 08:14:00,Legal has been assessing the risks of doing bl...,robert.badeer@enron.com murray.o neil@enron.co...
1,66,2000-08-03 02:56:00,Attached is a spreadsheet to estimate export f...,kim.ward@enron.com robert.badeer@enron.com mur...
2,74,2000-08-15 05:37:00,Kevin/Bob: Here is a quick rundown on the cons...,robert.badeer@enron.com john.massey@enron.com ...
3,80,2000-08-20 14:12:00,check this out and let everyone know what s up...,robert.badeer@enron.com jeff.richter@enron.com
4,83,2000-08-22 08:17:00,Further to your letter to us (addressed to Mr....,pgillman@schiffhardin.com kamarlantes@calpx.co...


In [5]:
test_set.head()

,sender,mids
0,karen.buckley@enron.com,298389 332383 298390 284071 366982 81773 81791...
1,amr.ibrahim@enron.com,48260 48465 50344 48268 50330 48237 189979 189...
2,andrea.ring@enron.com,366364 271168 271172 271167 271189
3,sylvia.hu@enron.com,134931 134856 233549 233517 134895 233584 3736...
4,phillip.platter@enron.com,274220 274225 274215 274223 274214 274207 2742...


In [6]:
test_info.head()

,mid,date,body
0,1577,2001-11-19 06:59:51,Note: Stocks of heating oil are very high for...
1,1750,2002-03-05 08:46:57,"Kevin Hyatt and I are going for ""sghetti"" at S..."
2,1916,2002-02-13 14:17:39,This was forwarded to me and it is funny. - Wi...
3,2094,2002-01-22 11:33:56,I will be in to and happy to assist too. I ma...
4,2205,2002-01-11 07:12:19,Thanks. I needed a morning chuckle.


In [7]:
training_df.head()

,mid,sender,date,body,recipients
0,47361,enron_update@concureworkplace.com,0001-08-26 22:16:36,The following reports have been waiting for yo...,kimberly.watson@enron.com
1,47362,enron_update@concureworkplace.com,0001-08-27 22:21:02,The following reports have been waiting for yo...,kimberly.watson@enron.com
2,47363,enron_update@concureworkplace.com,0001-08-28 22:25:35,The following reports have been waiting for yo...,kimberly.watson@enron.com
3,45909,enron_update@concureworkplace.com,0001-09-13 22:24:08,Employee Name: Kimberly WatsonReport Name: E...,kimberly.watson@enron.com
4,82030,enron_update@concureworkplace.com,0001-09-17 09:24:00,The following expense report is ready for appr...,barry.tycholiz@enron.com


In [8]:
test_df.head()

,mid,sender,date,body
0,284098,jonathan.mckay@enron.com,2001-11-02 05:25:29,"How is everyone.....mother, child.........fath..."
1,272008,dutch.quigley@enron.com,2001-11-02 05:34:55,-----Original Message-----From: \tWesner-Soon...
2,49273,james.d.steffes@enron.com,2001-11-02 05:57:55,Janine -Ok for you to cover the whole country....
3,71901,kim.ward@enron.com,2001-11-02 06:10:47,when?
4,82354,barry.tycholiz@enron.com,2001-11-02 06:17:44,WOW.... I am positive that your beautiful wife...


# Cross validation

### First way : temporal on all emails

In [9]:
if False :
    #Temporal separation between training and test
    # already time ordered
    pct_train = 0.8
    idx_separation = round(pct_train * training_info.shape[0])
    mid_train = training_info['mid'][:idx_separation]
    mid_test = training_info['mid'][idx_separation:]

    # Separation in training_info
    training_info_train = training_info[:idx_separation]
    training_info_test = training_info[idx_separation:]

    #Separation in training_set
    training_set_train = training_set.copy()
    training_set_test = training_set.copy()

    for row in range(training_set_train.shape[0]):
        mids = training_set_train['mids'][row].split(' ')
        new_mid = list(set(mids) & set(mid_train))
        training_set_train['mids'][row] = ' '.join(new_mid)

    for row in range(training_set_test.shape[0]):
        mids = training_set_test['mids'][row].split(' ')
        new_mid = list(set(mids) & set(mid_test))
        training_set_test['mids'][row] = ' '.join(new_mid)  
        
    #dataframe
    training_df_train = transform_dataset(training_info_train, training_set_train)
    training_df_test = transform_dataset(training_info_test, training_set_test)    

Some users are in the test set and NOT in the train set : mike.carson@enron.com, vkaminski@aol.com and christina.valdez@enron.com !


### Second way : pure randomness

In [9]:
if True:
    training_all_senders = training_set['sender'].get_values()

In [10]:
if True : 
    # Separation in training_df
    training_df_train, training_df_test = train_test_split(training_df, test_size=0.2, random_state=0)

    training_df_train = training_df_train.reset_index(drop=True)
    training_df_test = training_df_test.reset_index(drop=True)

    mid_train = training_df_train['mid']
    mid_test = training_df_test['mid']

In [11]:
if True:
    # Construction of _set with for each sender, the list of emails

    #Training
    training_set_train = pd.DataFrame()
    for sender in tqdm(training_all_senders):
        mids = training_df_train[training_df_train['sender'] == sender]['mid'].values.tolist()
        mids = ' '.join(list(map(str, mids)))
        training_set_train_sender = pd.DataFrame({'sender':[sender], 'mids': [mids]})

        training_set_train = training_set_train.append(training_set_train_sender)

    #reorder the columns
    training_set_train = training_set_train.reindex_axis(['sender', 'mids'], axis = 1)

    #add clean index
    training_set_train = training_set_train.reset_index(drop=True)

    #Test
    training_set_test = pd.DataFrame()
    for sender in tqdm(training_all_senders):
        mids = training_df_test[training_df_test['sender'] == sender]['mid'].values.tolist()
        mids = ' '.join(list(map(str, mids)))
        training_set_test_sender = pd.DataFrame({'sender':[sender], 'mids': [mids]})

        training_set_test = training_set_test.append(training_set_test_sender)

    #reorder the columns
    training_set_test = training_set_test.reindex_axis(['sender', 'mids'], axis = 1)

    #add clean index
    training_set_test = training_set_test.reset_index(drop=True)

In [12]:
if True:
    # Construction of _info with info about each mail

    #Training
    mids_train = training_df_train.mid.get_values().astype('str')
    training_info_train = training_info[(training_info['mid']).isin(mids_train)]

    #Test
    mids_test = training_df_test.mid.get_values().astype('str')
    training_info_test = training_info[(training_info['mid']).isin(mids_test)]

### third way : temporal by user

In [14]:
if False:
    training_all_senders = training_set['sender'].get_values()

In [15]:
if False:
    def split_data_v3(training_df, training_all_senders, pct_train = 0.8):
        #Initialisation of the two dataframe for learning and testing
        training_df_train = pd.DataFrame()
        training_df_test = pd.DataFrame()

        #We fill the values by taking the oldest emails in the train and the newest in test dataframe
        for sender in tqdm(training_all_senders):
            #dataframe with all emails sent by the sender
            training_df_sender = training_df[training_df['sender'] == sender] #non void
            limit = round(pct_train * training_df_sender.shape[0])

            training_df_train_sender = training_df_sender[:limit]
            training_df_test_sender = training_df_sender[limit:]

            training_df_train = training_df_train.append(training_df_train_sender)
            training_df_test = training_df_test.append(training_df_test_sender)

        #put the right index
        training_df_train = training_df_train.reset_index(drop=True)
        training_df_test = training_df_test.reset_index(drop=True)

        return training_df_train, training_df_test

    training_df_train, training_df_test = split_data_v3(training_df, training_all_senders, pct_train = 0.8)

In [16]:
if False:
    # Construction of _set with for each sender, the list of emails

    #Training
    training_set_train = pd.DataFrame()
    for sender in tqdm(training_all_senders):
        mids = training_df_train[training_df_train['sender'] == sender]['mid'].values.tolist()
        mids = ' '.join(list(map(str, mids)))
        training_set_train_sender = pd.DataFrame({'sender':[sender], 'mids': [mids]})

        training_set_train = training_set_train.append(training_set_train_sender)

    #reorder the columns
    training_set_train = training_set_train.reindex_axis(['sender', 'mids'], axis = 1)

    #add clean index
    training_set_train = training_set_train.reset_index(drop=True)

    #Test
    training_set_test = pd.DataFrame()
    for sender in tqdm(training_all_senders):
        mids = training_df_test[training_df_test['sender'] == sender]['mid'].values.tolist()
        mids = ' '.join(list(map(str, mids)))
        training_set_test_sender = pd.DataFrame({'sender':[sender], 'mids': [mids]})

        training_set_test = training_set_test.append(training_set_test_sender)

    #reorder the columns
    training_set_test = training_set_test.reindex_axis(['sender', 'mids'], axis = 1)

    #add clean index
    training_set_test = training_set_test.reset_index(drop=True)

In [17]:
if False:
    # Construction of _info with info about each mail

    #Training
    mids_train = training_df_train.mid.get_values().astype('str')
    training_info_train = training_info[(training_info['mid']).isin(mids_train)]

    #Test
    mids_test = training_df_test.mid.get_values().astype('str')
    training_info_test = training_info[(training_info['mid']).isin(mids_test)]

### Sanity check on cross validation

Let's make some sanity test : 

In [13]:
## == Check on the number of email in each part ====
n_senders = 125

def sanity_check_n_emails(dataset_df, dataset_info, dataset_set, name):
    n_mails_dataset_df = dataset_df.shape[0]
    n_mails_dataset_info = dataset_info.shape[0]
    n_mails_dataset_set = np.sum([len(dataset_set.mids[i].split(' ')) for i in range(n_senders)])
    print('SANITY CHECK ON ' + name)
    print('Number of emails in the full dataframe : %.0f \t _info : %.0f \t _set : %.0f \n' 
          %(n_mails_dataset_df, n_mails_dataset_info, n_mails_dataset_set))

#In the training (train + test)
sanity_check_n_emails(training_df, training_info, training_set, "training")

#In the train
sanity_check_n_emails(training_df_train, training_info_train ,training_set_train , "training for train")

#In the test
sanity_check_n_emails(training_df_test, training_info_test ,training_set_test, "training for test")


SANITY CHECK ON training
Number of emails in the full dataframe : 43613 	 _info : 43613 	 _set : 43613 

SANITY CHECK ON training for train
Number of emails in the full dataframe : 34890 	 _info : 34890 	 _set : 34890 

SANITY CHECK ON training for test
Number of emails in the full dataframe : 8723 	 _info : 8723 	 _set : 8723 



# Some handy structures on the train of cv

In [14]:
# Construct the address book for each sender, take 2/3 minutes
address_books_train = get_address_books(training_info_train, training_set_train)

In [15]:
# save all unique sender names
emails_ids_per_sender_train = get_mids_per_sender(training_set_train)
all_senders_train = emails_ids_per_sender_train.keys()

# save all unique recipient names    
all_recs_train = list(set([elt[0] for sublist in address_books_train.values() for elt in sublist]))

# save all unique user names 
all_users_train = []
all_users_train.extend(all_senders_train)
all_users_train.extend(all_recs_train)
all_users_train = list(set(all_users_train))

In [16]:
#An example
emails_ids_per_sender_train['enron_update@concureworkplace.com'][:5]

['190543', '190623', '42575', '40843', '190737']

In [17]:
def get_mids_sender_to_recipient(address_books, dataset_df):
    '''
    Computes the list of mids for each couple (sender, recipient)
    '''
    #a new dictionnary
    mids_sender_to_recipient = {}
    
    #all the senders
    all_senders = address_books_train.keys()
    
    for sender in tqdm(all_senders):
        #the mails sent by the sender
        mask_sender = (sender == dataset_df.sender)
        sub_dataset_df = dataset_df[mask_sender]
        sub_dataset_df = sub_dataset_df.reset_index(drop=True)
        
        rec_out = address_books[sender]
        for rec, out in rec_out: 

            #we search amongst them, those sent to the recipient
            mask_recipient = [rec in sub_dataset_df.recipients[i] for i in range(sub_dataset_df.shape[0])]

            #we save the result
            mids_sender_to_recipient[(sender, rec)] = sub_dataset_df['mid'][mask_recipient].values.tolist()
        
    return mids_sender_to_recipient 

In [18]:
# Takes 2/3 minutes
mids_sender_to_recipient = get_mids_sender_to_recipient(address_books_train, training_df_train)

In [19]:
#An example
mids_sender_to_recipient['enron_update@concureworkplace.com', 'mark.whitt@enron.com'][:5]

['28043', '27895', '27898', '27985', '28076']

In [20]:
#Mids of mails received by each senders
def get_mids_received_by_user(dataset_df):
    emails_ids_per_sender = {} 

    #iterate on each mail
    for row in tqdm(range(dataset_df.shape[0])):
        mid = dataset_df['mid'][row]
        recipients = dataset_df['recipients'][row].split(' ')

        for rec in recipients:
            if rec in emails_ids_per_sender.keys():
                emails_ids_per_sender[rec] = emails_ids_per_sender[rec] + [mid]
            else:
                emails_ids_per_sender[rec] = [mid]
                
    return emails_ids_per_sender            

In [21]:
#Mails received by user
mids_received_by_user_train = get_mids_received_by_user(training_df_train)

# Prédiction

## Random predictions on all users

In [22]:
# HERE THE PREDICTION WILL ONLY DEPENDS ON THE SENDER !
def predictions_random_per_sender(dataset_set, all_users, k=10):

    # will contain email ids, predictions for random baseline
    predictions_random_per_sender = {}

    for index, row in dataset_set.iterrows():
        name_ids = row.tolist() #sender + all ones mails)
        sender = name_ids[0]

        predictions_random_per_sender[sender] = random.sample(all_users, k)
        
    return predictions_random_per_sender    

In [23]:
#Make some predictions
predictions_random = predictions_random_per_sender(training_set_train, all_users_train)

#Fill the dataframe
training_df_test['random_pred'] = None
for row in range(training_df_test.shape[0]): #for each mail
    sender = training_df_test['sender'][row]

    #random prediction
    training_df_test['random_pred'][row] = predictions_random[sender] 

In [24]:
training_df_test.head()

,mid,sender,date,body,recipients,random_pred
0,337773,chris.germany@enron.com,2000-03-08 10:05:00,Looks good.Stephanie Sever 03/08/2000 05:35 ...,stephanie.sever@enron.com,"[javier.li@enron.com, downes.andrew@enron.com,..."
1,112428,susan.scott@enron.com,2001-05-04 12:02:26,"oh, it s on!",adriane.schultea@enron.com misti.day@enron.com...,"[dutch.quigley@enron.com, rick.suttle@enron.co..."
2,362746,chris.dorland@enron.com,2000-11-13 03:25:00,George has Oyster Creek coming back Nov. 25. W...,ricardo.perez@enron.com,"[philip.conn@enron.com, carlos.choudri@csfb.co..."
3,333249,c..giron@enron.com,2000-07-24 03:17:00,Last Friday I was trying to get into my accoun...,onepass@coair.com,"[roncarroll@bracepatt.com, kimberly.rizzi@enro..."
4,384123,eric.bass@enron.com,2000-08-17 06:53:00,"I want to trade you a 4th, 6th and 8th round p...",lqcolombo@aol.com,"[david.frost@enron.com, tomas.szamosvolgyi@enr..."


## Random predictions on the address book

In [25]:
# HERE THE PREDICTION WILL ONLY DEPENDS ON THE SENDER !

def predictions_random_address_book_per_sender(dataset_set, address_books, k=10):
    '''
    Recommends recipients randomly amongst the addresss books of the sender
    The number of recipients may be under k if the sender has not sent enough emails.
    '''
    # will contain email ids, predictions for random baseline
    predictions_random_address_book_per_sender = {}
    for index, row in dataset_set.iterrows():
        name_ids = row.tolist() #sender + all ones mails)
        sender = name_ids[0]

        recipients_of_the_sender = [elt[0] for elt in address_books[sender]]
        n_recipients_of_the_sender = len(recipients_of_the_sender)
        
        predictions_random_address_book_per_sender[sender] = random.sample(recipients_of_the_sender, min(k,n_recipients_of_the_sender))
    return predictions_random_address_book_per_sender    

In [26]:
#Make some predictions
predictions_random_address_book = predictions_random_address_book_per_sender(training_set_train, address_books_train)

#Fill the dataframe
training_df_test['random_pred_address_book'] = None
for row in range(training_df_test.shape[0]): #for each mail
    sender = training_df_test['sender'][row]

    #random prediction
    training_df_test['random_pred_address_book'][row] = predictions_random_address_book[sender]    

## Frequency based predictions

In [27]:
# HERE THE PREDICTION WILL ONLY DEPENDS ON THE SENDER !
def predictions_frequency_per_sender(dataset_set, address_books, k = 10):

    # will contain email ids, predictions for frequency baseline
    predictions_frequency_per_sender = {}

    for index, row in dataset_set.iterrows():
        name_ids = row.tolist() #sender + all ones mails
        sender = name_ids[0]

        # select k most frequent recipients for the user
        predictions_frequency_per_sender[sender] = [elt[0] for elt in address_books[sender][:k]]
        
    return predictions_frequency_per_sender

In [28]:
#Make some predictions
predictions_frequency = predictions_frequency_per_sender(training_set_train, address_books_train)

#Fill the dataframe
training_df_test['freq_pred'] = None
for row in range(training_df_test.shape[0]): #for each mail
    sender = training_df_test['sender'][row]

    #random prediction
    training_df_test['freq_pred'][row] = predictions_frequency[sender]    

## Weighted frequency-based predictions

In [29]:
def get_weighted_score(dataset_df):
    '''
    Computes a score between the sender and the recipient
    score(sender, recipient) = sum_{emails from sender to recipients} 1/(number of recipients in the email)
    
    == Input ==
    a dataframe with all the data
    == Output ==
    a dictionnary where the key is the sender and the value the tuple (recipient, score)
    '''
    #Learning phase
    dict_weight = {} #the key is (sender, recipient)
    for row in tqdm(range(dataset_df.shape[0])):

        sender = dataset_df['sender'][row]
        recipients = dataset_df['recipients'][row].split(' ')
        n_recipients = len(recipients)

        for rec in recipients: #for each recipients
            key = sender, rec
            if key in dict_weight : 
                dict_weight[key] =  dict_weight[key] + 1./n_recipients
            else:
                dict_weight[key] = 1./n_recipients

    #The right order
    weight_sender_recipient = {} #the key is (sender)
    for key, value in dict_weight.items():
        sender = key[0]
        rec = key[1]

        if sender in weight_sender_recipient:
            weight_sender_recipient[sender] = sorted(weight_sender_recipient[sender] + [(rec,value)], key=operator.itemgetter(1), reverse = True)      
        else:
            weight_sender_recipient[sender] = [(rec,value)]
            
    return dict_weight, weight_sender_recipient      

In [30]:
dict_weight, weight_sender_recipient = get_weighted_score(training_df_train)

In [31]:
#Make some predictions
training_df_test['weighted_freq_pred'] = None
k=10

#Testing phase
for row in range(training_df_test.shape[0]): #for each mail
    sender = training_df_test['sender'][row]

    #random prediction
    training_df_test['weighted_freq_pred'][row] = [elt[0] for elt in weight_sender_recipient[sender][:k]]

## Looks for a name

In [32]:
def get_recipients_from_name(name, all_users):
    '''
    Gives 0 or 1 recipient email address whose name is the one given is the input
    
    == Input ==
    name : a string with the name one is looking for
    all_users : a list of names, where the name is looked for, ordered by preference / score
    == Output ==
    if name is found, an email address
    '''
    
    if((len(name) > 3)): #remove 'it'
        for rec in all_users : 
            rec_name = rec.split('@')[0] #the part before the @
            score = str.find(rec_name, name.lower()) #returns -1 if the name is not in the email address, otherwise its position
            if score > -1: #a corresponding email address has been found
                return [rec]
        return [] #no corresponding email address
    return [] #the given name is too short 

In [33]:
get_recipients_from_name('Mark', all_users_train)

['mark.mcclure@enron.com']

In [34]:
#Make some predictions
training_df_test['name_freq_pred'] = None
k=10

#Testing phase
for row in tqdm(range(training_df_test.shape[0])): #for each mail
    sender = training_df_test['sender'][row]
    name_rec = training_df_test['body'][row].split(' ')[:1][0]

    #remove some nasty character
    #name_rec = str(name_rec).replace(',', '') 
    
    #find the possible recipients from this first word, amongst those the sender has already sent an email
    address_book_sender = [elt[0] for elt in address_books_train[sender]] #ordered by the frequency ! 
    
    recipients_from_name = get_recipients_from_name(name_rec, address_book_sender)

    #add the frequent recipients
    recipients_from_frequency = predictions_frequency[sender]
    
    #cumul
    result = recipients_from_name
    result.extend(x for x in recipients_from_frequency if x not in result)

    training_df_test['name_freq_pred'][row] = result[:k]


## Bag of words with TF-IDF

In [165]:
# TOO LONG

#early optimization is the root of all evil
k=10
#Make some predictions
training_df_test['tfidf_centroid_pred'] = None

for row in tqdm(range(training_df_test.shape[0])): #for each mail
    # == Search for some metadata and data ====
    
    # Search for the sender
    sender = training_df_test['sender'][row]
    
    #Search for all his previous emails sent (content + mid)
    corpus = list(training_df_train[training_df_train['sender'] == sender]['body']) #we may have some doublons

    #mids = emails_ids_per_sender_train[sender] #we have to keep them
    mids = list(training_df_train[training_df_train['sender'] == sender]['mid'])
    
  
    # == Clean the documents ====
    corpus_clean = {}
    for i, doc in enumerate(corpus):
        lowers = doc.lower()
        corpus_clean[mids[i]] = lowers

    # == TFIDF ====
    #corpus
    tfidf_matrix_sender = tfidf.fit_transform(corpus_clean.values())

    # new document
    new_doc = training_df_test['body'][row]
    response = tfidf.transform([new_doc])
    response
    
    # == Centroids ====
        
    #We search the centroids for each recipient of sender
    centroid_recipient = {} #contains for each recipient its vector representation

    for recipient, out in address_books_train[sender]:
        mids_sender_recipient = mids_sender_to_recipient[sender, recipient]
        mask_mid = [mids[i] in list(mids_sender_recipient) for i in range(len(mids))]

        centroid_recipient[recipient] = np.sum(tfidf_matrix_sender.todense()[mask_mid], axis = 0)
    
    # == Similarity ====
    
    #We calcultate the score for each recipient (cosine similarity) 
    score_recipient = {}
    for recipient, centroid in centroid_recipient.items():
        score_recipient[recipient] = float(cosine_similarity(centroid,response)[0])
    
    sorted_score_recipient = sorted(score_recipient.items(), key=operator.itemgetter(1), reverse = True)
    
    # == Prediction ====
    training_df_test['tfidf_centroid_pred'][row] = [elt[0] for elt in sorted_score_recipient[:k]]
    
    break

  0%|                                                 | 0/8723 [00:00<?, ?it/s]

chris.germany@enron.com
1968
1968
20


C:\Users\Mehdi\Annexes\PythonENSAE_v2\python\lib\site-packages\numpy\matrixlib\defmatrix.py:318: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  out = N.ndarray.__getitem__(self, index)


## Bayesian on email content

In [48]:
#Count the occurence of a word in all the data
print("Joining ...")
start_time = time.time()
all_bodies_train = ' '.join(training_df_train['body'])
taken_time = time.time() - start_time
print("\t %.3fs" %taken_time)

print("Lowering ...")
start_time = time.time()
lowers_train = all_bodies_train.lower()
taken_time = time.time() - start_time
print("\t %.3fs" %taken_time)

print("Tokenizing ...")
start_time = time.time()
tokens_train = tokenizer.tokenize(lowers_train)
taken_time = time.time() - start_time
print("\t %.3fs" %taken_time)

print("Steming ...")
start_time = time.time()
stemmed = stem_tokens(tokens_train)
taken_time = time.time() - start_time
print("\t %.3fs" %taken_time)

print("Counting ...")
start_time = time.time()
count_words_all_bodies_train = Counter(stemmed)
taken_time = time.time() - start_time
print("\t %.3fs" %taken_time)

print("Filtering ...")
start_time = time.time()
#filtered_tokens_train = [w for w in lowers_train if not w in stopwords.words('english')]
for sw in stopwords.words('english'):
    if sw in count_words_all_bodies_train.keys():
        del count_words_all_bodies_train[sw]
taken_time = time.time() - start_time
print("\t %.3fs" %taken_time)

Joining ...
	 0.277s
Lowering ...
	 0.078s
Tokenizing ...
	 5.592s
Steming ...
	 88.646s
Counting ...
	 1.778s
Filtering ...
	 0.565s


In [49]:
#Count all the unique words (stem without stop words) in all the training train
n_words_all_bodies_train = np.sum([value for key, value in count_words_all_bodies_train.items()])

In [50]:
def count_words(body):
    #Lowering
    lower = body.lower()
    
    #Tokenizing
    token = tokenizer.tokenize(lower)
    
    #Steming
    stemmed = stem_tokens(token)

    #Counting
    count_words = Counter(stemmed)
   
    #Filtering
    for sw in stopwords.words('english'):
        if sw in count_words.keys():
            del count_words[sw]
            
    #Sum of counts
    n_words = np.sum([value for key, value in count_words.items()])
    
    return count_words, n_words

In [51]:
def clean_body(body):
    lower = body.lower()
    token = tokenizer.tokenize(lower)
    stemmed = stem_tokens(token)
    return ' '.join(stemmed)

<font color = 'orange'> **Question :** </font>Should we consider all the emails shared between S and R or only those sent by S to R for $\mathbb{P}(w |R,S)$ ? option 1

In [81]:
sender

'c..giron@enron.com'

In [86]:
mids_sender_to_recipient[sender, rec]

['337771',
 '337990',
 '336843',
 '347881',
 '338090',
 '338288',
 '337838',
 '338088',
 '337880',
 '338525',
 '338103',
 '338501',
 '338201',
 '337885',
 '337978',
 '338777',
 '338062',
 '337861',
 '161798',
 '338608',
 '338096',
 '338237',
 '338252',
 '338392',
 '338534',
 '338193',
 '338044',
 '338487',
 '337977']

In [88]:
#TOOOOOOO LOONG

#early optimization is the root of all evil
k=10
# quid parameters => use the one given by the paper
lmbd = 0.6
gamma = 0.2
beta = 0.2

#We use a stochastic approach : max(25%, 100) mids

#Make some predictions
training_df_test['bayes_mail'] = None
N = 10 #training_df_test.shape[0]
## WARNING : We use a subset for the moment

for row in range(N): #for each mail
    start_time = time.time()
    sender = training_df_test['sender'][row]
    print(sender)
    body = training_df_test['body'][row]
    cleaned_body = clean_body(body)
    n_words_body = len(cleaned_body.split(' '))
    taken_time = time.time() - start_time
    #print('Accès aux données : %.3fs' %(taken_time))
    
    start_time = time.time()
    #Recipients considered : those whom already receive a mail from the sender
    recipients = [elt[0] for elt in address_books_train[sender]] # to test for a next version : (TODO) all_users_train
    taken_time = time.time() - start_time
    #print('Trouver les recipients possibles : %.3fs' %(taken_time))
    
    score_recipients = {}
    
    # == Compute P(w) ====
    #already done =)
    
    start_time = time.time()
    #mails received and sent by the sender
    mids_sent_sender = emails_ids_per_sender_train[sender]
    if sender in mids_received_by_user_train.keys(): #if the sender has received at least one mail
        mids_received_sender = mids_received_by_user_train[sender]
        all_mids_sender = list(set(mids_received_sender + mids_sent_sender)) #do not count double self mail
    else: #the sender sent but never received ...
        all_mids_sender = mids_sent_sender
        
    #We choose some mids
    n_mids_sender = np.minimum(np.maximum(100, round(0.25*len(all_mids_sender))), len(all_mids_sender))
    all_mids_sender = list(np.random.choice(all_mids_sender, n_mids_sender, replace=False))
    
    taken_time = time.time() - start_time
    #print('Trouver les mails s&r du sender : %.3fs' %(taken_time))
    ## WARNING :We use a subset for the moment

    min_recipients = min(10, len(recipients))
    for rec in recipients[:min_recipients]:
        print('\t', rec)
        # == Compute P(w|R) ====
        
        start_time = time.time()
        #search for all mids of received (and sent if they exist) mails
        mids_received_rec = mids_received_by_user_train[rec]
        if rec in all_senders_train:
            mids_sent_rec = emails_ids_per_sender_train[rec] #works only if the rec is also a sender
            all_mids_rec = list(set(mids_received_rec + mids_sent_rec)) #do not count double self mail
        else : 
            all_mids_rec = mids_received_rec
            
        #We choose some mids
        n_mids_rec = np.minimum(np.maximum(100, round(0.25*len(all_mids_rec))), len(all_mids_rec))
        all_mids_rec = list(np.random.choice(all_mids_rec, n_mids_rec, replace=False))
    
        taken_time = time.time() - start_time
        #print('Trouver les mids reçus par le rec : %.3fs' %(taken_time))
    
        start_time = time.time()
        #take all body of these mids
        bodies_rec = ' '.join(training_df_train[(training_df_train['mid']).isin(all_mids_rec)]['body'])
        count_words_rec, n_words_rec = count_words(bodies_rec)        
        taken_time = time.time() - start_time
        #print('Compter les mots dans les textes reçus et envoyés par le rec : %.3fs' %(taken_time))
    
        # == Compute P(w|R,S) ====
        start_time = time.time()
        #search for all mids exchange between the sender and the recipient
        all_mids_sender_to_rec = mids_sender_to_recipient[sender, rec]
        if rec in all_senders_train: #if the rec has sent mail TO THE SENDER !s
            all_mids_rec_to_sender = mids_sender_to_recipient[rec, sender]
            all_mids_sender_rec = list(set(all_mids_sender_to_rec + all_mids_rec_to_sender))
        else:
            all_mids_sender_rec = all_mids_sender_to_rec
        #all_mids_sender_rec = list(set(all_mids_rec + all_mids_sender))
        taken_time = time.time() - start_time
        #print('Trouver les mails envoyés et/ou reçus par le sender et le rec : %.3fs' %(taken_time))
    
        start_time = time.time()
        #take all body of these mids
        bodies_sender_rec = ' '.join(training_df_train[(training_df_train['mid']).isin(all_mids_sender_rec)]['body'])
        count_words_sender_rec, n_words_sender_rec = count_words(bodies_sender_rec)
        taken_time = time.time() - start_time
        #print('Compter les mots dans le corpus sender+rec : %.3fs' %(taken_time))
        
        start_time = time.time()
        # == Compute the score ====
        score = 0
        ## WARNING : We use a sum instead of a product otherwise the score is 0
        
        for word in cleaned_body.split(' '):
            #print('word', word, '\t SR', count_words_sender_rec[word], '\t R', count_words_rec[word], '\t all', count_words_all_bodies_train[word], '\t score', score)
            #print('sub : lmbd', count_words_sender_rec[word]/n_words_sender_rec, '\t gamma', count_words_rec[word]/n_words_rec, '\t beta', count_words_all_bodies_train[word]/n_words_all_bodies_train)
            score += lmbd * count_words_sender_rec[word]/n_words_sender_rec +\
            gamma * count_words_rec[word]/n_words_rec +\
            beta * count_words_all_bodies_train[word]/n_words_all_bodies_train
        
        score_recipients[rec] = score
        taken_time = time.time() - start_time
        #print('Calculer le score pour un rec : %.3fs' %(taken_time))
    
    sorted_score_recipients = sorted(score_recipients.items(), key=operator.itemgetter(1), reverse = True)
    training_df_test['bayes_mail'][row] = [key for key,value in sorted_score_recipients[:k]]

chris.germany@enron.com
	 scott.goodell@enron.com
	 victor.lamadrid@enron.com
	 brenda.fletcher@enron.com
	 robert.allwein@enron.com
	 judy.townsend@enron.com
	 joann.collins@enron.com
	 dick.jenkins@enron.com
	 dan.junek@enron.com
	 beverly.beaty@enron.com
	 victoria.versen@enron.com
susan.scott@enron.com
	 jeffery.fawcett@enron.com
	 lorraine.lindberg@enron.com
	 steven.harris@enron.com
	 ehillegeist@hotmail.com
	 ted.noble@enron.com
	 monique.sanchez@enron.com
	 mary.miller@enron.com
	 kevin.hyatt@enron.com
	 lisa.gillette@enron.com
	 michelle.lokay@enron.com
chris.dorland@enron.com
	 dan.dorland@enron.com
	 rlaird@natsource.ca
	 mmolloy@oebi.com
	 kdorland@titanelectric.com
	 bgabrielson@cdnam.com
	 ddorland@smednet.com
	 kim.melodick@enron.com
	 migeorge@deloitte.ca
	 mike.cowan@enron.com
	 keith.holst@enron.com
c..giron@enron.com
	 carole.frank@enron.com
	 kristi.giron@cfisd.net
	 dgiron1@pdq.net
	 jason.wolfe@enron.com


KeyError: ('jason.wolfe@enron.com', 'c..giron@enron.com')

In [97]:
mids_sender_to_recipient[rec, sender]

KeyError: ('jason.wolfe@enron.com', 'c..giron@enron.com')

In [84]:
sorted_score_recipients

[('sylvia.campos@enron.com', 0.55105088847236361),
 ('mary.franklin@enron.com', 0.50585441700903844),
 ('scott.neal@enron.com', 0.50383260386359963),
 ('brenda.fletcher@enron.com', 0.50026374475220559),
 ('sherry.anastas@enron.com', 0.49611748989650784),
 ('robert.allwein@enron.com', 0.49590089555674738),
 ('joan.veselack@enron.com', 0.49263073449156203),
 ('david.oliver@enron.com', 0.48991475444619098),
 ('steve.gillespie@enron.com', 0.48343162975078174),
 ('joe.casas@enron.com', 0.4808259311265588),
 ('katherine.kelly@enron.com', 0.47910407521466869),
 ('mark.breese@enron.com', 0.47892588096941269),
 ('alvin.thompson@enron.com', 0.47846733432987387),
 ('darla.saucier@enron.com', 0.47560218744287691),
 ('jessie.patterson@enron.com', 0.47371871889252232),
 ('sabra.dinari@enron.com', 0.47323510707556604),
 ('chris.germany@enron.com', 0.46891930510311713),
 ('marlene.hilliard@enron.com', 0.46796567268535),
 ('molly.lafuze@enron.com', 0.46769835872981708),
 ('dick.jenkins@enron.com', 0.46

In [85]:
training_df_test.head()

,mid,sender,date,body,recipients,random_pred,random_pred_address_book,freq_pred,weighted_freq_pred,name_freq_pred,bayes_mail
0,337773,chris.germany@enron.com,2000-03-08 10:05:00,Looks good.Stephanie Sever 03/08/2000 05:35 ...,[stephanie.sever@enron.com],"[javier.li@enron.com, downes.andrew@enron.com,...","[troy.denetsosie@enron.com, gloria.barkowsky@e...","[scott.goodell@enron.com, victor.lamadrid@enro...","[scott.goodell@enron.com, jim.homco@enron.com,...","[scott.goodell@enron.com, victor.lamadrid@enro...","[sylvia.campos@enron.com, mary.franklin@enron...."
1,112428,susan.scott@enron.com,2001-05-04 12:02:26,"oh, it s on!","[adriane.schultea@enron.com, misti.day@enron.c...","[dutch.quigley@enron.com, rick.suttle@enron.co...","[ashleastu@aol.com, emily.boon@msdw.com, davis...","[jeffery.fawcett@enron.com, lorraine.lindberg@...","[ted.noble@enron.com, ehillegeist@hotmail.com,...","[jeffery.fawcett@enron.com, lorraine.lindberg@...",None
2,362746,chris.dorland@enron.com,2000-11-13 03:25:00,George has Oyster Creek coming back Nov. 25. W...,[ricardo.perez@enron.com],"[philip.conn@enron.com, carlos.choudri@csfb.co...","[ptrussell@acurasouthwest.com, dawn.doucet@enr...","[dan.dorland@enron.com, rlaird@natsource.ca, m...","[dan.dorland@enron.com, rlaird@natsource.ca, m...","[migeorge@deloitte.ca, dan.dorland@enron.com, ...",None
3,333249,c..giron@enron.com,2000-07-24 03:17:00,Last Friday I was trying to get into my accoun...,[onepass@coair.com],"[roncarroll@bracepatt.com, kimberly.rizzi@enro...","[shona.wilson@enron.com, scott.t.crowell@us.cg...","[carole.frank@enron.com, kristi.giron@cfisd.ne...","[carole.frank@enron.com, dgiron1@pdq.net, kris...","[carole.frank@enron.com, kristi.giron@cfisd.ne...",None
4,384123,eric.bass@enron.com,2000-08-17 06:53:00,"I want to trade you a 4th, 6th and 8th round p...",[lqcolombo@aol.com],"[david.frost@enron.com, tomas.szamosvolgyi@enr...","[daphneco64@bigplanet.com, faith.killen@enron....","[bryan.hull@enron.com, matthew.lenhart@enron.c...","[shanna.husser@enron.com, jason.bass2@compaq.c...","[bryan.hull@enron.com, matthew.lenhart@enron.c...",None


# Test the P@10

In [35]:
#Split the recipients to use kdd_mapk
for row in range(training_df_test.shape[0]):
    training_df_test['recipients'][row] = training_df_test['recipients'][row].split(' ')

In [36]:
# MAP@10 for random predictions
mapk(training_df_test['recipients'], training_df_test['random_pred'])

0.00059133944379846008

In [37]:
# MAP@10 for random predictions among the address book of each sender
mapk(training_df_test['recipients'], training_df_test['random_pred_address_book'])

0.035544609675171736

In [38]:
# MAP@10 for frequency-based predictions
mapk(training_df_test['recipients'], training_df_test['freq_pred'])

0.28965128918339306

In [39]:
# MAP@10 for weighted-frequency-based predictions
mapk(training_df_test['recipients'], training_df_test['weighted_freq_pred'])

0.28366836937240913

In [40]:
# MAP@10 for name_frequency-based predictions
mapk(training_df_test['recipients'], training_df_test['name_freq_pred'])

0.29346892476722686

In [41]:
# MAP@10 for TF-IDF centroids predictions (withour STEM ! for the moment)
#mapk(training_df_test['recipients'], training_df_test['tfidf_centroid_pred'])

Let's check that in the dataframe :

In [57]:
#Score for each prediction with random recommendation
training_df_test['score_random'] = None

for row in range(training_df_test.shape[0]): #for each mail
    training_df_test['score_random'][row] = apk(training_df_test['recipients'][row], training_df_test['random_pred'][row], k=10)

In [58]:
#Score for each prediction with frequency-based recommendations 
training_df_test['score_freq'] = None

for row in range(training_df_test.shape[0]): #for each mail
    training_df_test['score_freq'][row] = apk(training_df_test['recipients'][row], training_df_test['freq_pred'][row], k=10)    

# Application on the real test dataset

In [59]:
test_df = transform_dataset(test_info, test_set)

In [60]:
test_df.head()

,mid,sender,date,body
0,284098,jonathan.mckay@enron.com,2001-11-02 05:25:29,"How is everyone.....mother, child.........fath..."
1,272008,dutch.quigley@enron.com,2001-11-02 05:34:55,-----Original Message-----From: \tWesner-Soon...
2,49273,james.d.steffes@enron.com,2001-11-02 05:57:55,Janine -Ok for you to cover the whole country....
3,71901,kim.ward@enron.com,2001-11-02 06:10:47,when?
4,82354,barry.tycholiz@enron.com,2001-11-02 06:17:44,WOW.... I am positive that your beautiful wife...


# Write 

In [62]:
#Prediction according to the frequency and the name (first word in the mail)


with open(path_to_results + 'predictions_name_freq_.txt', 'wb') as my_file:
    my_file.write(bytes('mid,recipients' + '\n', 'UTF-8'))
    for row in range(test_df.shape[0]):
        mid = test_df['mid'][row]
        name_freq_preds = test_df['name_freq_pred'][row]
        my_file.write(bytes(str(mid) + ',' + ' '.join(name_freq_preds) + '\n', 'UTF-8'))

KeyError: 'name_freq_pred'